In [80]:
import  pandas as pd
from pathlib import Path
from os.path import exists

In [81]:
root = Path("./..")
cleaned_data = root / "cleaned_data"
sentiments_climate_file = cleaned_data / "Sentiment_cleaned_data_climate.parquet"
speeches_file = cleaned_data / "combined_data_2013_2022_per_paragraph.parquet"

In [82]:
sentiments_climate = pd.read_parquet(sentiments_climate_file)
speeches = pd.read_parquet(speeches_file)

In [83]:
merged_info = speeches.merge(sentiments_climate, left_on="text_split", right_on="Document", how="left")

In [84]:
sentiments_climate = merged_info.drop(["corruption_scores", "corruption_sources", "corruption_standard_errors", "fos_yearly_evolution", "global_score", "Document"], axis=1)

In [85]:
sentiments_climate_per_year_mean_together = sentiments_climate.groupby(["iso_3", "year"], as_index=False)["sentiment"].mean()

In [86]:
paragraphs = sentiments_climate.groupby(["iso_3", "year"], as_index=False, observed=True)["text_split"].count()

In [87]:
paragraphs_sentiment = sentiments_climate.groupby(["iso_3", "year"], as_index=False)["sentiment"].count()

In [88]:
amount_of_time_spent_on_climate = paragraphs[paragraphs['text_split'] > 0].merge(paragraphs_sentiment, on=["iso_3", "year"])
amount_of_time_spent_on_climate["amount_of_time_spent_on_climate"] = amount_of_time_spent_on_climate.apply(lambda x: x["sentiment"] / x["text_split"], axis=1)

In [89]:
amount_of_time_spent_on_climate = amount_of_time_spent_on_climate.rename(columns={"text_split": "total_amount_paragraphs", "sentiment":"amount_climate_change_paragraphs"})

In [91]:
sentiments_climate_per_year_mean_together.merge(amount_of_time_spent_on_climate, on=['iso_3', 'year']).to_parquet(cleaned_data / "climate_change_sentiment_and_proportion_talked_about_climate_change.parquet")